In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/sdk-feature-store.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/sdk-feature-store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>

## 概观

这个Colab介绍了Vertex AI Feature Store，这是一种托管的云服务，供机器学习工程师和数据科学家存储、提供、管理和共享大规模的机器学习特征。

本Colab假设您了解基本的Google Cloud概念，如[项目](https://cloud.google.com/storage/docs/projects)、[存储](https://cloud.google.com/storage)和[Vertex AI](https://cloud.google.com/vertex-ai/docs)。一些机器学习知识也有帮助，但不是必需的。

### 数据集

这个Colab在所有会话中使用一个电影推荐数据集作为示例。任务是训练一个模型来预测用户是否会观看一部电影，并在线提供这个模型。

### 目标

在这个笔记本中，您将学习如何：

    * 创建特征存储、实体类型和特征资源。
    * 将您的特征导入Vertex AI Feature Store。
    * 使用导入的特征提供在线预测请求。
    * 访问离线作业中导入的特征，如训练作业。

### 成本

本教程使用Google Cloud的可计费组件：

* Vertex AI
* Cloud Storage
* Cloud BigQuery

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Google Cloud笔记本**，您的环境已经满足运行本笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用 Python 3 的虚拟环境中运行的 Jupyter notebook

Google Cloud 的[设置 Python 开发环境指南](https://cloud.google.com/python/setup)和[Jupyter 安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简明的一组说明：

1. [安装并初始化 Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装 Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。

4. 要安装 Jupyter，请在终端 shell 中的命令行中运行 `pip install jupyter`。

5. 要启动 Jupyter，请在终端 shell 的命令行中运行 `jupyter notebook`。

6. 在 Jupyter Notebook 仪表板中打开本笔记本。

在你开始之前

### 安装额外的包

对于这个Colab，您需要安装Python的Vertex SDK。

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip install {USER_FLAG} --upgrade google-cloud-aiplatform

### 重新启动内核

安装SDK后，您需要重新启动笔记本内核以便找到这些包。您可以从 *Kernel -> Restart Kernel* 重新启动内核，或者运行以下命令：

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置您的Google Cloud项目

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得300美元的免费信用额度，可用于支付计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您正在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`前缀开头的行作为shell命令运行，并将以`$`前缀开头的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
print("Project ID: ", PROJECT_ID)

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud记事本**，您的环境已经经过身份验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按提示进行oAuth身份验证。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**将此服务帐号授予项目访问权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，然后选择**Vertex AI管理员**。在过滤框中输入"Storage Object Admin"，然后选择**Storage Object Admin**。

5. 点击**创建**。包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐号密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 导入库并定义常量

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import Feature, Featurestore

REGION = "[your-region]"  # @param {type:"string"}
FEATURESTORE_ID = "movie_prediction"
INPUT_CSV_FILE = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv"
ONLINE_STORE_FIXED_NODE_COUNT = 1

aiplatform.init(project=PROJECT_ID, location=REGION)

## 术语和概念

### 功能存储数据模型

Vertex AI Feature Store以以下3个重要的层次概念组织数据：
```
功能存储 -> 实体类型 -> 功能
```
* **功能存储**: 存储功能的地方
* **实体类型**: 在功能存储下，实体类型描述要建模的对象，可以是真实的也可以是虚拟的。
* **功能**: 在实体类型下，功能描述实体类型的属性

在电影预测示例中，您将创建一个名为`movie_prediction`的功能存储。该存储有2个实体类型：`users`和`movies`。 `users`实体类型具有`age`， `gender` 和 `liked_genres`功能。`movies`实体类型具有`titles`， `genres` 和 `average rating`功能。

创建特征存储并定义模式

创建特征存储

创建特征存储的方法返回一个长时间运行的操作（LRO）。 LRO启动一个异步任务。 对于其他API方法，例如更新或删除特征存储，也会返回LRO。 运行代码单元将创建一个特征存储并打印过程日志。

In [ ]:
fs = Featurestore.create(
    featurestore_id=FEATURESTORE_ID,
    online_store_fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT,
    project=PROJECT_ID,
    location=REGION,
    sync=True,
)

使用下面的函数调用来检索一个特征存储，并检查它是否已创建。

In [ ]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

### 创建实体类型

可以在特征存储类内创建实体类型。下面，创建用户实体类型和电影实体类型。将会打印出一个流程日志。

In [ ]:
# Create users entity type
users_entity_type = fs.create_entity_type(
    entity_type_id="users",
    description="Users entity",
)

In [ ]:
# Create movies entity type
movies_entity_type = fs.create_entity_type(
    entity_type_id="movies",
    description="Movies entity",
)

要检索实体类型或检查它是否已创建，请在Featurestore对象上使用[get_entity_type](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L106)或[list_entity_types](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L278)方法。

In [ ]:
users_entity_type = fs.get_entity_type(entity_type_id="users")
movies_entity_type = fs.get_entity_type(entity_type_id="movies")
print(users_entity_type)
print(movies_entity_type)

In [ ]:
fs.list_entity_types()

### 创建特征
可以在每个实体类型中创建特征。通过以下方法向用户实体类型和电影实体类型添加定义特征。

In [ ]:
# to create features one at a time use
users_feature_age = users_entity_type.create_feature(
    feature_id="age",
    value_type="INT64",
    description="User age",
)

users_feature_gender = users_entity_type.create_feature(
    feature_id="gender",
    value_type="STRING",
    description="User gender",
)

users_feature_liked_genres = users_entity_type.create_feature(
    feature_id="liked_genres",
    value_type="STRING_ARRAY",
    description="An array of genres this user liked",
)

使用[list_features](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/entity_type.py#L349)方法列出给定实体类型的所有特征。

In [ ]:
users_entity_type.list_features()

In [ ]:
movies_feature_configs = {
    "title": {
        "value_type": "STRING",
        "description": "The title of the movie",
    },
    "genres": {
        "value_type": "STRING",
        "description": "The genre of the movie",
    },
    "average_rating": {
        "value_type": "DOUBLE",
        "description": "The average rating for the movie, range is [1.0-5.0]",
    },
}

In [ ]:
movie_features = movies_entity_type.batch_create_features(
    feature_configs=movies_feature_configs,
)

## 搜索创建的特征

虽然 `list_features` 方法允许您轻松查看单个实体类型的所有特征，但 Feature 类中的 [search](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/feature.py#L352) 方法可在给定位置（如 `us-central1`）跨所有特征存储和实体类型进行搜索，并返回特征列表。这可以帮助您发现其他人创建的特征。

您可以基于特征属性进行查询，包括特征 ID、实体类型 ID 和特征描述。您还可以通过筛选特定的特征存储、特征值类型或标签来限制结果。以下是一些搜索示例。

使用下面的代码片段搜索特征存储中的所有特征。

In [ ]:
my_features = Feature.search(query="featurestore_id={}".format(FEATURESTORE_ID))
my_features

现在，将搜索范围缩小到类型为“DOUBLE”的特性。

In [ ]:
double_features = Feature.search(
    query="value_type=DOUBLE AND featurestore_id={}".format(FEATURESTORE_ID)
)
double_features[0].gca_resource

或者，将搜索结果限制在带有特定关键词的ID和类型特征上。

In [ ]:
title_features = Feature.search(
    query="feature_id:title AND value_type=STRING AND featurestore_id={}".format(
        FEATURESTORE_ID
    )
)
title_features[0].gca_resource

导入特征值

在您可以将特征值用于在线/离线服务之前，您需要先导入特征值。在这一步中，您将学习如何通过从GCS（Google Cloud Storage）中摄取值来导入特征值。我们也可以从BigQuery或Pandas dataframe中导入特征值。

### 源数据格式和布局

支持将BigQuery表/Avro/CSV作为输入数据类型。无论您使用什么格式，每个导入的实体都 *必须* 有一个ID；此外，每个实体可以 *选择性地* 具有一个时间戳，指定特征值生成的时间。这个Colab使用Avro作为输入，在这个公共 [bucket](https://console.cloud.google.com/storage/browser/cloud-samples-data-us-central1/vertex-ai/feature-store/datasets) 上。Avro模式如下：

**对于用户实体**：
```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
```

**对于电影实体**：
```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### 为用户实体类型导入特征值

在导入时，在您的请求中指定以下内容：

* 要导入的特征的ID
* 数据源URI
* 数据源格式：BigQuery Table/Avro/CSV

In [ ]:
USERS_FEATURES_IDS = [feature.name for feature in users_entity_type.list_features()]
USERS_FEATURE_TIME = "update_time"
USERS_ENTITY_ID_FIELD = "user_id"
USERS_GCS_SOURCE_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
)
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(USERS_FEATURES_IDS)

In [ ]:
users_entity_type.ingest_from_gcs(
    feature_ids=USERS_FEATURES_IDS,
    feature_time=USERS_FEATURE_TIME,
    entity_id_field=USERS_ENTITY_ID_FIELD,
    gcs_source_uris=USERS_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    sync=False,
)

### 导入电影实体类型的特征值

同样地，将电影实体类型的特征值导入到特征存储中。

In [ ]:
MOVIES_FEATURES_IDS = [feature.name for feature in movies_entity_type.list_features()]
MOVIES_FEATURE_TIME = "update_time"
MOVIES_ENTITY_ID_FIELD = "movie_id"
MOVIES_GCS_SOURCE_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
)
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(MOVIES_FEATURES_IDS)

In [ ]:
movies_entity_type.ingest_from_gcs(
    feature_ids=MOVIES_FEATURES_IDS,
    feature_time=MOVIES_FEATURE_TIME,
    entity_id_field=MOVIES_ENTITY_ID_FIELD,
    gcs_source_uris=MOVIES_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    sync=False,
)

在线服务

在线提供允许您为少量实体提供特征数值。它专为对延迟敏感的服务设计，例如在线模型预测。例如，对于电影服务，您可能希望快速展示当前用户最有可能观看的电影。

使用Python SDK，轻松读取一个实体的特征值。默认情况下，SDK将返回每个特征的最新值，即具有最新时间戳的特征值。

要读取特征值，请指定实体类型ID和要读取的特征。默认情况下，将选择实体类型的所有特征。响应将输出并显示选定的实体类型ID和选定的特征值作为Pandas dataframe。

In [ ]:
users_entity_type.read(entity_ids="bob")

In [ ]:
movies_entity_type.read(entity_ids="movie_01", feature_ids="title")

每个请求读取多个实体

要从多个实体中读取特征值，请指定不同的实体类型ID。默认情况下，将选择实体类型的所有特征。请注意，由于该SDK对延迟敏感，建议在使用时仅获取少量实体。

In [ ]:
users_entity_type.read(entity_ids=["bob", "alice"])

In [ ]:
movies_entity_type.read(
    entity_ids=["movie_02", "movie_03", "movie_04"], feature_ids=["title, genres"]
)

现在您已经学会了如何为在线服务获取导入的特征值，下一步是学习如何将导入的特征值用于离线用例。

批量服务

批量服务用于获取大批量的特征数值以实现高吞吐量，通常用于训练模型或批量预测。在本节中，您将学习如何使用特征存储的批量服务功能为训练示例做准备。

### 用例

**任务**是准备一个训练数据集来训练一个模型，该模型可以预测给定用户是否会观看给定电影。要实现这一点，您需要两组输入：

* 特征：您已经导入到特征存储中。
* 标签：记录了用户X观看了电影Y的地面真实数据。

更具体地说，地面事实观察在表1中描述，所需的训练数据集在表2中描述。表2中的每一行是根据表1中的实体ID和时间戳从Vertex AI特征存储中导入的特征值的连接结果。在这个例子中，从用户中选择`年龄`、`性别`和`喜欢的类型`特征，从电影中选择`标题`、`类型`和`平均评分`特征来训练模型。请注意，表中只显示了正例，即您可以想象存在一个标签列，其值都为`True`。

[batch_serve_to_bq](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L770)以表1作为输入，从特征存储中连接所有必需的特征值，并返回用于训练的表2。

<h4 align="center">表1. 地面真实数据</h4>

用户 | 电影 | 时间戳            
----- | -------- | -------------------- 
爱丽丝  | 电影天堂     | 2019-11-01T00:00:00Z 
鲍勃  | 闪灵     | 2019-11-15T18:09:43Z 
...   | ...      | ...     


<h4 align="center">表2. 使用批处理服务生成的预期训练数据</h4>

时间戳            | 用户实体类型 | 年龄 | 性别 | 喜欢的类型 | 电影实体类型 | 标题 | 类型 | 平均评分  
-------------------- | ----------------- | --------------- | ---------------- | -------------------- | - | -------- | --------- | ----- 
2019-11-01T00:00:00Z | 鲍勃              | 35        | 男                | [动作, 犯罪]      | 电影_02            | 闪灵 | 恐怖 | 4.8 
2019-11-01T00:00:00Z | 爱丽丝             | 55        | 女                | [剧情, 喜剧]      | 电影_03          | 电影天堂 | 浪漫 | 4.5 | 
... | ... | ... | ... | ... | ... | ... | ... | ...

为什么要有时间戳？

请注意，表2中有一个名为`timestamp`的列。这表示地面真实数据观察时的时间。这是为了避免数据不一致。

例如，表2的第二行表示用户`alice`在`2019-11-01T00:00:00Z`观看了电影`意大利电影天堂`。特征存储保留了所有时间戳的特征值，但在批量服务期间*仅*在给定时间戳获取特征值。那天，Alice可能是54岁，但现在Alice可能已经是56岁；特征存储将`age=54`返回为Alice的年龄，而不是`age=56`，因为这是在观察时间点该特征的值。同样，其他特征也可能是随时间变化的，比如`liked_genres`。

### 为输出创建BigQuery数据集

您需要在`us-central1`中创建一个BigQuery数据集来托管输出数据。输入您想要创建的数据集的名称，并指定您希望存储稍后创建的输出的表的名称。这些将在下一节中使用。

**确保表名尚不存在**。

In [ ]:
from datetime import datetime

from google.cloud import bigquery

In [ ]:
# Output dataset
DESTINATION_DATA_SET = "movie_predictions"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

# Output table. Make sure that the table does NOT already exist; the BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "training_data"  # @param {type:"string"}

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

In [ ]:
# Create dataset
client = bigquery.Client(project=PROJECT_ID)
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

批量读取特征值

组装请求，指定以下信息：

* 标签数据在哪里，即表1。
* 要读取哪些特征，即表2中的列名。

输出存储在BigQuery表中。

In [ ]:
SERVING_FEATURE_IDS = {
    # to choose all the features use 'entity_type_id: ['*']'
    "users": ["age", "gender", "liked_genres"],
    "movies": ["title", "average_rating", "genres"],
}

In [ ]:
fs.batch_serve_to_bq(
    bq_destination_output_uri=DESTINATION_TABLE_URI,
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_uri=INPUT_CSV_FILE,
)

LRO完成后，您应该能够在[BigQuery控制台](https://console.cloud.google.com/bigquery)中看到结果，作为之前创建的BigQuery数据集下的新表。

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

您也可以保留项目，通过运行以下代码来删除特征存储和BigQuery数据集：

In [ ]:
# Delete Featurestore
fs.delete(force=True)

In [ ]:
# Delete BigQuery dataset
client = bigquery.Client(project=PROJECT_ID)
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))